# How are the technological hubs of manufacturing evolving in the world? An analysis on the rate of innovation. 
<h2 align="center"> Pre-Work</h2>

----

### Description of Section: Data cleaning and initial analysis

In this first part of the projects, I am aiming to analyse the quality of the data retrieved and saved in the [prework section](./01_retrieving_data_and_DB_creation.ipynb) to be able to carry out further analysis on it. 
This jupyter notebook includes the following steps:
1. Reading the files
2. Basic analysis techniques from the Pandas library to analyse the quality of the data
3. Exporting the data into csv. formats. 

### The Data:

I will be using a number of different data sources for the two different questions asked in the project description. 

##### 1. Innovation analysis 
For the first part of my project, **the innovation analysis**, I will be using information of patent registration, both [by country](https://www.wipo.int/ipstats/en/), as well as [by company](http://www.oecd.org/sti/intellectual-property-statistics-and-analysis.htm). Added to this I will also be using a number of indicators provided by the [World Bank Dataset](https://data.worldbank.org/topic/science-and-technology) to measure innovation. 

Specific datasets used, such as the Industry Classification Benchamark indices, where imported form a self made csv file. The information was extracted form [Wikipedia.](https://en.wikipedia.org/wiki/Industry_Classification_Benchmark)


##### 2. Market analysis
In the second part of the project,  I will attempt a **market analysis**, I will be using data from [Crunchbase](https://data.crunchbase.com/docs/open-data-map), as well as the database on patent regrstration from [JRC-OECD COR&DIP database v.1](http://www.oecd.org/sti/intellectual-property-statistics-and-analysis.htm), 2017  to do this. 


----


In [1]:
# importing basic libraries needed: 

# for database manipulation
import pandas as pd
import pandas_profiling as pp
import numpy as np
import os
from datetime import  datetime

# for visualisations:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

# for statistics
from scipy import stats

## Basis:
First the datasets that contain the primary keys will be created and modified accordingly to maintain a consistent nomeclature throught the project. 

This includes the country dataframe and the industry dataframe. 

The country dataframe is based on the World Bank report, and inlcudes:
- Country Code
- Country Name
- Income Group

The industry table, as mentioned above, provides from the  Industry Classification Benchamark indices. 

##### 1. Countries table:

In [233]:
# open file
country = pd.read_csv("../00_data/01_raw/patent_data/world_bank_countires.csv", index_col="Country Code")

In [234]:
# read first rows
country.head()

,Country Name,Region,IncomeGroup,SpecialNotes
Country Code,,,,
ABW,Aruba,Latin America & Caribbean,High income,NaN
AFG,Afghanistan,South Asia,Low income,NaN
AGO,Angola,Sub-Saharan Africa,Lower middle income,NaN
ALB,Albania,Europe & Central Asia,Upper middle income,NaN
AND,Andorra,Europe & Central Asia,High income,NaN


In [235]:
# drop specialNotes as its not useful 
country.drop("SpecialNotes", axis=1, inplace=True)

In [236]:
# change names of columns for consistency:
country = country.rename(columns = {"Country Name":"country name", "Region":"region", "IncomeGroup":"income_group"})
country.head()

,country name,region,income_group
Country Code,,,
ABW,Aruba,Latin America & Caribbean,High income
AFG,Afghanistan,South Asia,Low income
AGO,Angola,Sub-Saharan Africa,Lower middle income
ALB,Albania,Europe & Central Asia,Upper middle income
AND,Andorra,Europe & Central Asia,High income


In [237]:
# save for later use
country.to_csv("../00_data/00_clean_datasets/country_pk.csv") 

As we have to change some of the observations' country code, I will create a df with both types (2 letters, and 3 letters) of index to replace inconsistencies. 

For this I will take the code naming from one of the other tables, and match them to our ISO3 code.

In [238]:
# selecting and opening file with country names and 2 letter country code
df = pd.read_csv("../00_data/01_raw/patent_data/patents_by_country_and_technology.csv")

In [239]:
# selecting unique country code
code_lst = df["Origin (Code)"].unique()

In [240]:
# selecting unique country names
country_lst = df["Origin"].unique()

In [241]:
# make into dataframe 
data_tuples = list(zip(country_lst,code_lst))
df = pd.DataFrame(data_tuples, columns=['country name','country code2'])
df.head()

,country name,country code2
0,Afghanistan,AF
1,Albania,AL
2,Algeria,DZ
3,Andorra,AD
4,Angola,AO


In [242]:
# merge into country table to be able to replace later:
# need to reset index for country code to be recognised as new index
country.reset_index(inplace=True)
country.head()
country_2c= country.merge(df, how="left", on="country name")

In [244]:
country_2c.drop(["region", "income_group"], axis=1, inplace=True)

In [251]:
country_2c = country_2c.rename(columns = {"Country Code":"country code"})

In [252]:
country_2c.head()

,country code,country name,country code2
0,ABW,Aruba,AW
1,AFG,Afghanistan,AF
2,AGO,Angola,AO
3,ALB,Albania,AL
4,AND,Andorra,AD


country_2c can now be used to replace the country names, all columns but country code, country name, country code 2 can be dropped

##### 2. Industries table:
Industry in ICB (industry classification benchamerk) format, needs to be changed to better understand 

In [155]:
# open file and read
industry = pd.read_csv("../00_data/01_raw/patent_data/ICBs.csv", index_col="ICB")
industry.head()

,industry
ICB,
530,Oil & Gas Producers
570,"Oil Equipment, Services & Distribution"
580,Alternative Energy
1350,Chemicals
1730,Forestry & Paper


In [156]:
# reset index for future merge on ICB column in later table
industry_in = industry.reset_index()
industry_in.head()

,ICB,industry
0,530,Oil & Gas Producers
1,570,"Oil Equipment, Services & Distribution"
2,580,Alternative Energy
3,1350,Chemicals
4,1730,Forestry & Paper


In [157]:
# save to clean data Industry table:
industry.to_csv("../00_data/00_clean_datasets/industry_pk.csv") 

## 1. Innovation analysis

##### Patents by country

In [253]:
# open file
patents_by_industry = pd.read_csv("../00_data/01_raw/patent_data/patents_by_country_and_technology.csv")

In [254]:
# read
patents_by_industry.head()

,Origin,Origin (Code),Field of technology,1980,1981,1982,1983,1984,1985,1986,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Afghanistan,AF,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,Afghanistan,AF,"1 - Electrical machinery, apparatus, energy",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN
2,Afghanistan,AF,2 - Audio-visual technology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AF,5 - Basic communication processes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
4,Afghanistan,AF,6 - Computer technology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,1.0,2.0,2.0,1.0,5.0,6.0,5.0,5.0


In [255]:
# fill nans with 0 to complete table, the rest of the table will be left like this to use in further analysis
patents_by_industry = patents_by_industry.fillna(0)

In [256]:
# change names of columns for consistency 
patents_by_industry = patents_by_industry.rename(columns = {"Origin":"country name", "Origin (Code)":"country", "Field of technology":"industry"})

I need to change the country codes to ISO 3 for consistency.

Steps:

In [257]:
# change country indexing to 3 letter naming for consistency:
# merge in inofrmation
patents_by_industry= patents_by_industry.merge(country_2c, how="left", on="country name")

In [260]:
# drop all not needed cols 
patents_by_industry.drop(["country", "country code2"], axis=1, inplace=True)

In [267]:
# change back order of columns:
patents_by_industry = patents_by_industry[['country name', 'country code', 'industry', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']]

In [268]:
# changing the industry label for consistency:
patents_by_industry["industry"].value_counts()

35 - Civil engineering                         161
33 - Furniture, games                          159
29 - Other special machines                    156
32 - Transport                                 156
27 - Engines, pumps, turbines                  154
1 - Electrical machinery, apparatus, energy    153
16 - Pharmaceuticals                           152
19 - Basic materials chemistry                 151
13 - Medical technology                        149
23 - Chemical engineering                      149
25 - Handling                                  144
18 - Food chemistry                            143
34 - Other consumer goods                      143
10 - Measurement                               143
6 - Computer technology                        143
Unknown                                        141
14 - Organic fine chemistry                    139
26 - Machine tools                             139
12 - Control                                   138
30 - Thermal processes and appa

The industry naming not consistent with current naming defined in the Primary Key table. However, the categories are too different for it to be directly translatable. 

For the moment we are going to leave it like this, in case I face problems with this in the analysis, I will retrace my cleaning steps. 

In [269]:
# save initial table to .csv or a table containing country patents per industry
patents_by_industry.to_csv("../00_data/00_clean_datasets/Q1/patents_by_industry.csv") 

In [274]:
# grouping by country for sum of patents per country:
patents_by_country = patents_by_industry.groupby("country code").sum()
patents_by_country.head()

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
country code,,,,,,,,,,,,,,,,,,,,,
ABW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,2.0,4.0,2.0,3.0,7.0,0.0,2.0,5.0,0.0
AFG,0.0,0.0,1.0,2.0,3.0,0.0,1.0,2.0,1.0,7.0,...,73.0,37.0,29.0,17.0,22.0,49.0,54.0,84.0,79.0,100.0
AGO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0,0.0
ALB,6.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,13.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,1.0
AND,1.0,2.0,2.0,1.0,2.0,1.0,3.0,4.0,1.0,3.0,...,7.0,15.0,14.0,3.0,15.0,6.0,18.0,6.0,12.0,5.0


In [275]:
# save data for future use
patents_by_country.to_csv("../00_data/00_clean_datasets/patents_by_country.csv") 

#####  Innovation rate of countries

In [276]:
innovation_markers = pd.read_csv("../00_data/01_raw/patent_data/world_bank_innovation_rate.csv")

In [277]:
innovation_markers.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,ABW,High-technology exports (% of manufactured exp...,TX.VAL.TECH.MF.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,3.213858,3.292313,4.052655e+00,1.024010e+01,4.915729e+00,5.449211e+00,4.694768e+00,3.758736e+00,5.380172e+00,NaN
1,Aruba,ABW,High-technology exports (current US$),TX.VAL.TECH.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,866862.000000,560538.000000,1.073498e+06,3.325655e+06,1.627152e+06,1.663353e+06,1.194859e+06,1.158612e+06,1.654810e+06,NaN
2,Aruba,ABW,Technicians in R&D (per million people),SP.POP.TECH.RD.P6,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aruba,ABW,Researchers in R&D (per million people),SP.POP.SCIE.RD.P6,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aruba,ABW,"Trademark applications, total",IP.TMK.TOTL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# fill nans with 0 to complete table
innovation_markers = innovation_markers.fillna(0)

In [40]:
# remove unnecesary colum: indicator code
innovation_markers.drop("Indicator Code", axis=1, inplace=True)

In [41]:
# change names of columns for consistency 
innovation_markers = innovation_markers.rename(columns = {"Country Name":"country name", "Country Code":"country", "Indicator Name":"indicator"})

In [80]:
# save to .csv
innovation_markers.to_csv("../00_data/00_clean_datasets/Q1/innovation_markers.csv") 

For the moment, this concludes the cleaning of this set, if later I realise that I need more grouping, I will change the table again. 

### Patents per company
Database documentation can be found in pdf format in the data folder

##### Company lst

In [284]:
# open file
company_lst = pd.read_csv("../00_data/01_raw/patent_data/2017-COR&DIP_Company_list.txt", sep="|")

In [285]:
# read
company_lst.head()

,Company_id,Company_name,Ctry_Code,Worldrank,ICB3,NACE2,ISIC4_STAN38
0,1,ZUMTOBEL,AT,851,2730,2740,27
1,2,ANDRITZ,AT,892,2750,2895,28
2,3,AUSTRIAMICROSYSTEMS,AT,1023,9570,2611,26
3,4,AUSTRIA TECHNOLOGIE & SYSTEMTECHNIK,AT,1173,2730,2612,26
4,5,VOESTALPINE,AT,596,1750,2452,24-25


The variables we need are: 

- Company_id*: Unique company identifier (from 1 to 2000)*
- Company_name: Company name, as listed in the 2015 Scoreboard
- Ctry_code: ISO2 country code
- ICB-3D: Industry sector, as listed in the 2015 Scoreboard


In [286]:
# drop unecessary columns
company_lst.drop(["NACE2", "ISIC4_STAN38", "Worldrank"], axis=1, inplace=True)

In [287]:
# make column names more understandable 
company_lst = company_lst.rename(columns = {"Company_name":"company", "Ctry_Code":"country code2", "ICB3":"ICB"})
company_lst.head()

,Company_id,company,country code2,ICB
0,1,ZUMTOBEL,AT,2730
1,2,ANDRITZ,AT,2750
2,3,AUSTRIAMICROSYSTEMS,AT,9570
3,4,AUSTRIA TECHNOLOGIE & SYSTEMTECHNIK,AT,2730
4,5,VOESTALPINE,AT,1750


In [288]:
# needs a change of country naming!!!

# change country indexing to 3 letter naming for consistency:
# merge in inofrmation
company_lst= company_lst.merge(country_2c, how="left", on="country code2")

In [290]:
# drop all not needed cols 
company_lst.drop(["country name", "country code2"], axis=1, inplace=True)

In [291]:
company_lst.head()

,Company_id,company,ICB,country code
0,1,ZUMTOBEL,2730,AUT
1,2,ANDRITZ,2750,AUT
2,3,AUSTRIAMICROSYSTEMS,9570,AUT
3,4,AUSTRIA TECHNOLOGIE & SYSTEMTECHNIK,2730,AUT
4,5,VOESTALPINE,1750,AUT


In [292]:
# merge ICB industry name of index
company_lst= company_lst.merge(industry_in, how="left", on="ICB").set_index("Company_id")

# drop ICB index column
company_lst.drop("ICB", axis=1, inplace=True)

# view changes
company_lst.head()

,company,country code,industry
Company_id,,,
1,ZUMTOBEL,AUT,Electronic & Electrical Equipment
2,ANDRITZ,AUT,Industrial Engineering
3,AUSTRIAMICROSYSTEMS,AUT,Technology Hardware & Equipment
4,AUSTRIA TECHNOLOGIE & SYSTEMTECHNIK,AUT,Electronic & Electrical Equipment
5,VOESTALPINE,AUT,Industrial Metals & Mining


As we will later on merge in the count of patents per company, we do not yet save to csv.

##### Company patents

In [58]:
company_patents = pd.read_csv("../00_data/01_raw/patent_data/2017-COR&DIP_Patent_Portfolio.txt", sep="|", index_col=0)

C:\Users\fabia hnt\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\fabia hnt\AppData\Roaming\Python\Python37\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [59]:
company_patents.head()

,Patent_appln_id,Publn_auth,Patent_publn_nr,Patent_filing_date,Inpadoc_family_id,Family_filing_date,IP5_2_offices
Company_id,,,,,,,
1,380646999,EP,2805106,2013-01-14,9215041,2012-01-16,0
1,380718081,EP,2807419,2013-01-22,9222565,2012-01-26,0
1,380385005,EP,2620923,2013-01-22,9222808,2012-01-27,0
1,380385015,EP,2620936,2013-01-22,9222811,2012-01-24,0
1,380889366,EP,2810535,2013-02-01,9232760,2012-02-03,1


In [60]:
# remove unecessary columns
lst = ["Patent_publn_nr", "Family_filing_date", "IP5_2_offices", "Inpadoc_family_id"]
company_patents.drop(lst, axis=1, inplace=True)

The variables we need are: 
- Company_id*: Unique company identifier*
- Patent_appln_id: Patent application identifier (Appln_id from PATSTAT, Autumn 2016)
    - This will be used as a count 
- Publn_auth: IP5 Offices (EP, JP, KR, US, CN)
- Patent_filing_date: Application date
    - The date in not really necessary for the main question, I will hower leave it in the table in case we want to use it in the future

In [61]:
# make column names more understandable
company_patents = company_patents.rename(columns = {"Publn_auth":"office", "Patent_filing_date":"date"})

In [62]:
company_patents.head()

,Patent_appln_id,office,date
Company_id,,,
1,380646999,EP,2013-01-14
1,380718081,EP,2013-01-22
1,380385005,EP,2013-01-22
1,380385015,EP,2013-01-22
1,380889366,EP,2013-02-01


In [63]:
#checkign attributes
company_patents.dtypes

Patent_appln_id     int64
office             object
date               object
dtype: object

In [64]:
# changing date into datetime 
company_patents["date"] =  pd.to_datetime(company_patents["date"], format="%Y/%m/%d")

In [66]:
# save as cleaned file for later use
company_patents.to_csv("../00_data/00_clean_datasets/company_patents.csv") 

To create a count, only one column with information is in needed, we will use the unique attribute Pattent applicaiton number.  The count is going to be mergerd into the company_lst table to have all data in one set

In [67]:
# count function
counts = company_patents.groupby(["Company_id", "office"]).count()[["Patent_appln_id"]]
counts = counts.rename(columns = {"Patent_appln_id":"count"})

This will be merged into the company list dataframe to obtain one table with all the inofmration for further manipulaiton of the set. 

In [68]:
counts.head()

count
Company_id office       
1          CN         84
           EP        263
           JP          1
           US         86
2          CN         74

##### Merging into one dataset for next analysis

In [293]:
# merge the count list of patents per office onto the company list. 
company_lst = company_lst.merge(counts, how="inner", left_index=True, right_index=True)

In [294]:
company_lst.head()

company country code                           industry  \
Company_id office                                                             
1          CN      ZUMTOBEL          AUT  Electronic & Electrical Equipment   
           EP      ZUMTOBEL          AUT  Electronic & Electrical Equipment   
           JP      ZUMTOBEL          AUT  Electronic & Electrical Equipment   
           US      ZUMTOBEL          AUT  Electronic & Electrical Equipment   
2          CN       ANDRITZ          AUT             Industrial Engineering   

                   count  
Company_id office         
1          CN         84  
           EP        263  
           JP          1  
           US         86  
2          CN         74

In [295]:
# saving cleaned table for later
company_lst.to_csv("../00_data/00_clean_datasets/Q1/company_lst.csv") 

## 2. Company analysis - market research

##### Company finance

In [83]:
# open file
company_info = pd.read_csv("../00_data/01_raw/patent_data/2017-COR&DIP_Company_financial.txt", sep="|", index_col=0)

In [84]:
# read
company_info.head()

,Year,RD,NS,CAPEX,OP,EMP
Company_id,,,,,,
1,2011,55.071,1280.312,57.159,34.591,7456.0
1,2012,66.926,1243.616,59.509,21.659,7162.0
1,2013,71.800,1246.831,65.553,12.144,7291.0
1,2014,89.739,1312.620,76.576,41.091,7234.0
2,2011,65.641,4595.993,76.974,263.445,16750.0


Varibales: 
- Year: 2011-2014
- RD: Research and Development investment (million €)
- NS: Net sales (million €)
- CAPEX: Capital expenditure (million €)
- OP: Operating profits (million €)
- EMP: Number of employees

In [85]:
# changing the names of variables for better understanding 
company_info = company_info.rename(columns = {"RD":"R&D_investemetn", "NS":"net_sales", "OP":"operating_profits", "EMP":"employees"})

In [86]:
# removing CAPEX as not really needed:
company_info.drop("CAPEX", axis=1, inplace=True)

In [87]:
# saving cleaned table for later
company_info.to_csv("../00_data/00_clean_datasets/Q2/company_info.csv") coujt

### BONUS: Organizations, financing and descriptions  (from 2015) 

In [116]:
organizations = pd.read_csv("../00_data/01_raw/crunchbase15/companies_15.csv")

In [117]:
organizations.head()

,name,category_list,status,country_code,city,founded_at,last_funding_at
0,#fame,Media,operating,IND,Mumbai,NaN,05/01/2015
1,:Qounter,Application Platforms|Real Time|Social Network...,operating,USA,Delaware City,04/09/2014,14/10/2014
2,"(THE) ONE of THEM,Inc.",Apps|Games|Mobile,operating,NaN,NaN,NaN,30/01/2014
3,0-6.com,Curated Web,operating,CHN,Beijing,01/01/2007,19/03/2008
4,004 Technologies,Software,operating,USA,Champaign,01/01/2010,24/07/2014


###### Checking for NaNs

As I will be sotring by category and founding data, all NaNs in those fields need to be removed, no matter what fraction of the data I loose. 
(I need all attributes but location to be filled for the data to be of use)

In [118]:
# checking for NaN
organizations.isna().sum()/len(organizations)*100

name                0.001507
category_list       4.743250
status              0.000000
country_code       10.483968
city               12.096191
founded_at         22.934245
last_funding_at     0.000000
dtype: float64

In [119]:
# we wil drop all NaNs but those in the location attribute
# we will drop city as we do not need it, coutnry is enough

organizations.drop("city", axis=1, inplace=True)

# create df for location to merge back in after drop nan
location = organizations[["country_code"]]

# drop country form original
organizations.drop("country_code", axis=1, inplace=True)

# drop nans 
organizations.dropna(inplace=True)

# merge countries back in
organizations = organizations.merge(location, how="left", left_index=True, right_index=True)

In [120]:
organizations.isna().sum()

name                  0
category_list         0
status                0
founded_at            0
last_funding_at       0
country_code       3183
dtype: int64

##### checking dataframe attributes and types

In [121]:
print(organizations.shape)

(49710, 6)


In [130]:
organizations.head()

,name,category_list,status,founded_at,last_funding_at,country_code
1,:Qounter,Application Platforms|Real Time|Social Network...,operating,04/09/2014,2014-10-14,USA
3,0-6.com,Curated Web,operating,01/01/2007,2008-03-19,CHN
4,004 Technologies,Software,operating,01/01/2010,2014-07-24,USA
6,Ondine Biomedical Inc.,Biotechnology,operating,01/01/1997,2009-12-21,CAN
7,H2O.ai,Analytics,operating,01/01/2011,2015-11-09,USA


In [123]:
#cheking df attributes
organizations["category_list"].value_counts()

Software                                                                               3137
Biotechnology                                                                          2431
E-Commerce                                                                              999
Mobile                                                                                  883
Curated Web                                                                             794
Clean Technology                                                                        731
Hardware + Software                                                                     699
Enterprise Software                                                                     663
Health Care                                                                             634
Games                                                                                   625
Advertising                                                                     

We see that we have a large variety of industry attribute types. 
As I will be sorting by industry later, this does not concearn me, as I can selct the industry I want with a simple np.where funciton. 

In [141]:
# checking for types for the time attributes
organizations.dtypes

name                       object
category_list              object
status                     object
founded_at         datetime64[ns]
last_funding_at    datetime64[ns]
country_code               object
dtype: object

In [125]:
# changing last_funding_at into datetime
organizations["last_funding_at"] = pd.to_datetime(organizations["last_funding_at"], format="%d/%m/%Y")

In [137]:
#changing founded_at into datetime
# error = "coerce" will create NaT values for those companies with different format. 
organizations["founded_at"] = pd.to_datetime(organizations["founded_at"], errors = 'coerce', format="%d/%m/%Y")

In [138]:
# check how many rows don't have the info anymore
organizations.isna().sum()

name                  0
category_list         0
status                0
founded_at           88
last_funding_at       0
country_code       3183
dtype: int64

In [139]:
# drop those rows as we did beforea s to not loose the country code info:
# drop country form original
organizations.drop("country_code", axis=1, inplace=True)

# drop nans 
organizations.dropna(inplace=True)

# merge countries back in
organizations = organizations.merge(location, how="left", left_index=True, right_index=True)

In [140]:
organizations.head()

,name,category_list,status,founded_at,last_funding_at,country_code
1,:Qounter,Application Platforms|Real Time|Social Network...,operating,2014-09-04,2014-10-14,USA
3,0-6.com,Curated Web,operating,2007-01-01,2008-03-19,CHN
4,004 Technologies,Software,operating,2010-01-01,2014-07-24,USA
6,Ondine Biomedical Inc.,Biotechnology,operating,1997-01-01,2009-12-21,CAN
7,H2O.ai,Analytics,operating,2011-01-01,2015-11-09,USA


In [142]:
# saving cleaned table for later
organizations.to_csv("../00_data/00_clean_datasets/Q2/crunchbase_organizations.csv")

## 3. Organising Database


### Description of section: 
- Importing Datasets
- Organising Database
- Creating Database connection and exporting Datasets to cloud


The prework of the Project consists basically on the organisation of the data storage and the creation of the tables that I will be using through this Analysis. 

Goal of this part is to describe the Database, upload the credentials needed to access it as a guest, and uploading the Data to the cloud. 

I will be using a number of different datasets for the analysis of the project.

The relationship and organisation of these can be seen below: 

![DB_diagram](../02_visualisations/Database_org.PNG)

> Note that the tables country and industry are only used as key-holders for extra information. I will not be analysing the informaiton in them directly. 

## 3. Exporing Data into cloud

In [145]:
# import special SQL librarbies
from sqlalchemy import create_engine

In [168]:
# opening connection to SQL:
driver = "mysql+pymysql:"
user = "ironhack"
password = "Ironhack1"
ip = "35.240.116.117"
database = "Project_5"

In [169]:
connection_string = f"{driver}//{user}:{password}@{ip}/{database}"
engine = create_engine(connection_string)

Tables to export:
- industry (ICB)
- innovation_markers (world bank)
- organizations (crunchbase)
- country (world bank)
- patents_by_industry (WIPO)

In [297]:
country.to_sql("Country", con=engine, if_exists="append", index=True)

In [298]:
industry.to_sql("Industry", con=engine, if_exists="append", index=True)

In [299]:
innovation_markers.to_sql("Innovation_Markers", con=engine, if_exists="append", index=True)

In [300]:
organizations.to_sql("Organizations", con=engine, if_exists="append", index=True)

In [301]:
patents_by_industry.to_sql("Patent_by_Country", con=engine, if_exists="append", index=True)

##### DataBase guest credentials:

<div class="alert alert-block alert-info">
<b>Credentials for DB:</b> 

##### To access the database, see the details below:
Note that this database has an IP restriction, and can therefore only be viewed in the IRONHACK campus. If the DB needs to be accessed from a different  IP address, please contact me. 

>**Connection information:** <br>
>**User name**: ironhack <br> 
>**Passowrd**: Ironhack1 <br>
>**Host name**: 35.240.116.117 <br>
</div>